In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
from pybedtools import BedTool
import pybedtools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

tf.logging.set_verbosity(tf.logging.INFO)

#if __name__ == "__main__":
#    tf.app.run()

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
def cnn_model(features, labels, mode):
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 4])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5, 5],
                             padding="same",activation=tf.nn.relu)
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(inputs=conv1,filters=64,kernel_size=[5, 5],
                             padding="same",activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    conv3 = tf.layers.conv2d(inputs=pool1, filters=64,kernel_size=[5,5],
                             padding="same",activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dense2 = tf.layers.dense(inputs=dense1, units=1024, activation=tf.nn.relu)
    dense3 = tf.layers.dense(inputs=dense2, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense3, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN )
    logits = tf.layers.dense( inputs=dropout, units=1 )
    
    #make predictions
    predictions = {
        "methylation": logits,
        #"Mean_Squared_Error": tf.metrics.mean_squared_error(labels,logits,name="MSE_tensor")
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec( mode=mode, predictions=predictions )
    
    total = tf.reduce_sum( tf.square ( tf.subtract( labels, tf.reduce_mean( labels ) ) ) )
    residual = tf.reduce_sum( tf.square( tf.subtract( labels, predictions["methylation"] ) ) )
    loss = tf.div(residual, total)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "Mean_squared_error": tf.div(tf.reduce_sum(tf.square(tf.subtract(labels, predictions["methylation"]))), 
                                     tf.reduce_sum(tf.square(tf.subtract(labels, tf.reduce_mean(labels)))))}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [16]:
def main(unused_argv):
    methy = pd.read_csv('Mouse_DMRs_methylation_level.csv',header=None)
    
    #apply bedtools to read fasta files 
    a = pybedtools.example_bedtool('/home/h5li/methylation_DMR/DMR_coordinates_extended_b500.bed')
    fasta = pybedtools.example_filename('/home/h5li/methylation_DMR/mm10.fasta')
    a = a.sequence(fi=fasta)
    seq = open(a.seqfn).read()
    
    
    #read and extract DNA sequences 
    DNA_seq_list = seq.split('\n')
    DNA_seq_list.pop()
    DNA_seq = []
    for index in range(len(DNA_seq_list)//2):
        DNA_seq.append(DNA_seq_list[index*2 + 1].upper())
    
    #chop DNA sequences to have same length
    Uni_DNA = []
    target_length = 1024
    for s in DNA_seq:
        if len(s) < target_length:
            print('Exceptions!')
        diff = len(s) - target_length
        if diff % 2 == 0:
            side = diff // 2
            Uni_DNA.append(s[side:-side])
        else:
            right = diff // 2
            left = diff// 2 + 1
            Uni_DNA.append(s[left:-right])
    
    DNA = []
    for u in Uni_DNA:
        sequence_vector = []
        for mode in ['A','C','G','T']:
            a = []
            for index in range(len(u)):
                if u[index] == mode:
                    a.append(float(1))
                else:
                    a.append(float(0))
            sequence_vector.append(a)
        DNA.append(np.array(sequence_vector))
    DNA = np.array(DNA)
    
    #shuffle the data
    train_data = DNA[:48000]
    test_data = DNA[48000:]
    
    cell_type = 5
    labels = pd.read_csv('Mouse_DMRs_methylation_level.csv',header = None)
    train_labels = labels[:48000][cell_type]
    test_labels = labels[48000:][cell_type]
    
    methylation_classifier = tf.estimator.Estimator(model_fn=cnn_model, model_dir="./CNN_results")
    
    # Set up logging for predictions
    tensors_to_log = {"Mean_Squared_Error": 'softmax_tensor'}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": train_data},y=train_labels,
                                                        batch_size=100,num_epochs=None,shuffle=True)
    
    methylation_classifier.train(input_fn=train_input_fn,steps=20000,hooks=[logging_hook])
    
    #Evaluate Results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": test_data},y=test_labels,num_epochs=1,shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [17]:
main(2)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_steps': None, '_is_chief': True, '_master': '', '_service': None, '_num_worker_replicas': 1, '_task_type': 'worker', '_model_dir': './CNN_results', '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_evaluation_master': '', '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_train_distribute': None, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4445538c88>, '_device_fn': None, '_session_config': None, '_save_summary_steps': 100}


INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_steps': None, '_is_chief': True, '_master': '', '_service': None, '_num_worker_replicas': 1, '_task_type': 'worker', '_model_dir': './CNN_results', '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_evaluation_master': '', '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_train_distribute': None, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4445538c88>, '_device_fn': None, '_session_config': None, '_save_summary_steps': 100}


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:loss = 767.5081774801395, step = 1


INFO:tensorflow:loss = 767.5081774801395, step = 1


ERROR:tensorflow:Model diverged with loss = NaN.


ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: NaN loss during training.